<!--
#  Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
#
#    Licensed under the Apache License, Version 2.0 (the "License").
#    You may not use this file except in compliance with the License.
#    You may obtain a copy of the License at
#
#        http://www.apache.org/licenses/LICENSE-2.0
#
#    Unless required by applicable law or agreed to in writing, software
#    distributed under the License is distributed on an "AS IS" BASIS,
#    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#    See the License for the specific language governing permissions and
#    limitations under the License.
-->

# Orcherstration notebook for building the lake
***End-to-End Orchestration for Building Out a Data Lake in Orbit Workbench***

---
---

## Content
1. [Introduction](#Orcherstration-notebook-for-building-the-lake)
2. [Set Up](#Set-Up)
 1. [Imports](#Imports)
 2. [Locate Bucket Paths](#Locate-Bucket-Paths)
 3. [Create Databases](#Create-Databases)
 4. [Get Parameters](#Get-Parameters)
3. [S3 Clean Up](#Step-2:-S3-Clean-Up)
4. [Extract Zip Files in Parallel](#Step-3:-Extract-Zip-Files-in-Parallel)
5. [Read the CSV Files and Create Glue tables with Parquet format according to schema](#Step-4:-Read-the-CSV-Files-and-Create-Glue-tables-with-Parquet-format-according-to-schema)
 1. [Connect to Spark and Access Cluster](#Connect-to-Spark-and-Access-Cluster)
 2. [Create Glue Tables](#Create-Glue-Tables)
 3. [Check Tables are Created](#Check-that-Glue-Tables-are-Created)

---

## Introduction
This notebook orchestrates the Data Lake creation. It performs end-to-end functionality starting with handling zipped csv data files and ultimately creating a data lake on AWS. In order to do so, we execute a set of notebooks for the different steps of creating the lake, including:

* **Example-2-Extract-Files** - Extracting Zip Files Data to a Target s3 Bucket in Parallel
* **Example-3-Load-Database-Athena** - Find File Schema and Create Glue Tables with Parquet Output

To successfully run this notebook, you must be the "Lake Creator" user in your environment. Give a look at the following steps for how to go about orchestrating your own data lake build and feel free to look at the other 2 notebooks to get a more in-depth understanding of the process!


## Set Up

#### Imports
First, let's import all of the modules we will need for building our Data Lake, including Spark EMR Cluster, JSON, etc. Lets store our session state so that we can create service clients to s3 and glue.

Next, lets define the location of our notebooks in s3 and check our team space (we **MUST** be the lake-creator to orchestrate our data lake!):


In [ ]:
import os
import sys
import boto3
import time
import json
from aws_orbit_sdk import controller
from aws_orbit_sdk.common import get_workspace
from pathlib import Path

!env | grep AWS

# import aws.utils.notebooks.controller as controller
# from aws.utils.notebooks.common import get_workspace
# import aws.utils.notebooks.spark.emr as sparkConnection
my_session = boto3.session.Session()
my_region = my_session.region_name
s3 = boto3.client('s3')
glue = boto3.client('glue')

In [ ]:
workspace = get_workspace()
notebook_bucket = workspace['scratch-bucket']
team_space = workspace['team_space']
workspace

#### Locate Bucket Paths
Now, let's use Amazon Systems Manager (SSM) to get the bucket names for our users, unsecured lake, and secured lake buckets 
(**Note:** we will use these bucket names to locate and store data later on):

In [ ]:
ssm = boto3.client('ssm')
def get_ssm_parameters(ssm_string, ignore_not_found=False):
    ssm = boto3.client('ssm')
    
    try:
        return json.loads(ssm.get_parameter(Name=ssm_string)['Parameter']['Value'])
    except Exception as e:
        if ignore_not_found:
            return {}
        else:
            raise e

        
def get_demo_configuration():
    return get_ssm_parameters("/orbit/demo", True)


demo_config = get_demo_configuration()
lake_bucket = demo_config.get("lake-bucket") or notebook_bucket
users_bucket = notebook_bucket

#lake_bucket = demo_config['lake_bucket']
#secured_lake_bucket = demo_config['secured_lake_bucket']
#users_bucket = demo_config['users_bucket']
#(lake_bucket,secured_lake_bucket,users_bucket)

#### Create Databases
We have to create 3 databases: 1 for our raw data, 1 for a default database (both on the lake_bucket path) and 1 for users in our user_bucket.

We can do so with the following function which:

  **1.** Deletes the existing database with a given name (if it exists)
  
  **2.** Create new one located in designated s3 bucket

In [ ]:
def create_db(name, location, description=''):
    try:
        response = glue.delete_database(
            Name=name
        )
    except:
        pass
    response = glue.create_database(
        DatabaseInput={
            'Name': name,
            'Description': description,
            'LocationUri': f's3://{location}/{name}'
        }
    )

In [ ]:
create_db('cms_raw_db', lake_bucket,'lake: claims data from cms')
create_db('default', lake_bucket)
create_db('users', users_bucket)

#### Get Parameters
Lastly, we set the source paths of our Zip files, a path to our extracted data folders, and our database name howing our raw data:

In [ ]:
location = glue.get_database(Name='cms_raw_db')['Database']['LocationUri']
bucket = location[5:].split('/')[0]
(bucket, location)

In [ ]:
sourcePrefix = "cms/"
sourceFolder = "landing/data/" + sourcePrefix
bucketName = bucket
extractedPrefix = "extracted/"
extractedFolder = "s3://{}/{}".format(bucketName,extractedPrefix)
database_name = "cms_raw_db"
team_space=workspace['team_space']

---
## Step 2: S3 Clean Up

Another step we must take before beginning to extract our data and orchestrate our data lake is to clean out our S3 files for any existing data files sitting there. Let's remove existing content in the following folders:
* Files stored in our Extracted Data Folder
* Data Stored in our 'cms_raw_db' table
* Remove Test Output so we can populate with new test results


In [ ]:
!aws s3 ls $extractedFolder
!aws s3 rm --recursive $extractedFolder
!aws s3 rm --recursive s3://$bucketName/landing
!aws s3 ls $extractedFolder

In [ ]:
!aws s3 ls s3://$bucketName/$database_name/
!aws s3 rm --recursive  s3://$bucketName/$database_name/
!aws s3 ls s3://$bucketName/$database_name/

***Lets Copy our Sample Data to our New s3 Bucket and Begin our Orchestration:***

In [ ]:
dm_s3_bucket = json.loads(ssm.get_parameter(
    Name=f'/orbit/{workspace.get("env_name")}/manifest'
)['Parameter']['Value'])['toolkit-s3-bucket']
dm_s3_bucket
!aws s3 sync s3://$dm_s3_bucket/data s3://$bucketName/landing/data

In [ ]:
!aws s3 cp s3://$dm_s3_bucket/cms/schema s3://$bucketName/landing/cms/schema/ --recursive

In [ ]:
def get_schemas(source_bucket_name, prefix='', suffix=''):
    s3 = boto3.resource("s3")
    bucket = s3.Bucket(name=source_bucket_name) 
    schemas = []
    for o in bucket.objects.all():
        if (o.key.startswith(prefix)):
            name = os.path.basename(o.key).split(".")[0]
            schemaStr = o.get()['Body'].read().decode('utf-8') 
            schema = json.loads(schemaStr) #StructType.fromJson(json.loads(schemaStr))
            schemas.append((name, schema))
    return schemas

def get_schema(schemas, filename):
    for (schema_name, schema) in schemas:
        #print(f"{schema_name} in {filename} : {schema_name in filename}")
        if schema_name in filename:
            return schema_name, schema
    return None, None

schemas = get_schemas(bucketName, 'landing/cms/schema/')

***
## Step 3: Extract Zip Files in Parallel

We have completed all of the necessary set up and s3 clean up and our ready to move into the first phase of our data lake orchestration. Here we will:

* Handle Zipped Files and Extract their CSV Data
* Migrate the Extracted Data back to s3 in a new Target Bucket
* Schedule Multiple Notebooks to Execute in Parallel

We will schedule separate notebooks to run **Example-2-Extract-Files** and execute in parallel. You can refer to that notebook which goes step-by-step handling zipped files and unzipping and migrating their content back to s3.

Here we will define error checking functions to assert that our extraction was successful. We ensure the number of executions matches what is expected and that there are no errors in our execution history:


In [ ]:
def run_file_extraction():
    notebooks = []
    for key in s3.list_objects_v2(Bucket=bucketName, Prefix=sourceFolder)['Contents']:
        file = key['Key']
        schema = get_schema(schemas, file)
        s3_data_folder = os.path.join(extractedFolder, schema[0] if schema[0] else "")
        notebook = {
          "notebookName": "Example-2-Extract-Files.ipynb",
          "sourcePath": "/efs/shared/samples/notebooks/A-LakeCreator",
          "targetPath": "/efs/shared/regression/notebooks/A-LakeCreator",
          "params": {
            "bucketName": bucketName,
            "zipFileName": file,
            "targetFolder": s3_data_folder,
            "use_subdirs" : False if schema[0] else True
          },
          "ExecutionType": "ecs"
        }
        notebooks.append(notebook)

    notebooksToRun = {
      "compute": {
          "container" : {
              "p_concurrent": "10"
          },
          "compute_type": "ecs",
          "node_type": "fargate"
      },
      "tasks":  notebooks  
    }
    # notebooks
    containers = controller.run_notebooks(notebooksToRun)
    print (containers)
    controller.wait_for_tasks_to_complete([containers], 60,10, False)


In [ ]:
extractedFolder

In [ ]:
def checkNotebooks(executions, expected_count):
    assert len(executions) == expected_count
    for index, row in executions.iterrows():
        if 'error@' in row['relativePath']:
            raise AssertionError('error in ' + row['relativePath'])
    print("SUCCESS")

In [ ]:
%%time

run_file_extraction()

In [ ]:
executions = controller.get_execution_history("/efs/shared/regression/notebooks/A-LakeCreator", "Example-2-Extract-Files.ipynb")
executions

In [ ]:
try: 
    checkNotebooks(executions, 8)
except AssertionError as e:
    print("Failed once, lets give one more try")
    !rm -rf /efs/shared/regression/notebooks/A-LakeCreator/Example-2*
    run_file_extraction()
    executions = controller.get_execution_history("/efs/shared/regression/notebooks/A-LakeCreator", "Example-2-Extract-Files.ipynb")
    checkNotebooks(executions,8)
    

***We will check our Output in our extractedFolder and split the output into an array of File Outputs:***

In [ ]:
%%bash --out output --err error -s $extractedFolder

aws s3 ls "$1"

In [ ]:
print(output)
files = output.split('\n')
print("total files: " + str(len(files)))


***
## Step 4: Read the CSV Files and Create Glue tables with Parquet format according to schema

Our Zipped files have been succesfully extracted and their CSV data content is placed in our **extractedFolder**. We now must collect the schema for our different data tables saved as csv files and create Parquet Output Glue Tables with set schema located in our target directory on s3.


#### Create Glue Tables
We will now create our Parquet formatted Glue Tables by scheduling and executing the notebook, **Example-3-Load-Database-Athena**. For each extracted csv data file in our extracted folder our notebook will perform the following:

* Find for each file the corresponding schema

* Read the file using spark

* Create external tables to create the Glue table and parquet output

You can refer to that notebook which goes step-by-step through the process of schema detection and table creation. We will execute the **run_glue_table_loading** on 4 concurrent containers and checks the execution history to ensure that the code ran for all the data files:

In [ ]:
load_data_notebook = "Example-3-Load-Database-Athena"

def run_glue_table_loading(concurrent_containers=4):
    containers = []
    found_schemas = []
    i = 0
    for key in s3.list_objects_v2(Bucket=bucketName, Prefix=extractedPrefix)['Contents']:
        file = key['Key']
        p = Path(file).parent
        schema = get_schema(schemas, file)
        if schema in found_schemas:
            continue
        i = i + 1
        print(f"Found schema: {schema[0]}")
        found_schemas.append(schema)
        notebooksToRun = {
          "compute": {
              "compute_type": "ecs",
              "node_type": "fargate",
              "container": {
                  "p_concurrent" :1
              }
          },
          "tasks":  [
                {
                      "notebookName": f"{load_data_notebook}.ipynb",
                      "sourcePath": "/efs/shared/samples/notebooks/A-LakeCreator",
                      "targetPath": "/efs/shared/regression/notebooks/A-LakeCreator",
                      "targetPrefix": "unsecured-{}".format(i),
                      "params": {
                            "source_bucket_name" : bucketName,
                            "target_bucket_name" : bucketName,
                            "database_name" : "cms_raw_db",
                            "schema_dir" : "landing/cms/schema",
                            "file_path": str(p),
                            "region": my_region
                      }      
                }
          ],
          "env_vars": [
                {
                    'name': 'AWS_ORBIT_S3_BUCKET',
                    'value': bucketName
                }
          ]
        }

        t = time.localtime()
        current_time = time.strftime("%H:%M:%S", t)

        container = controller.run_notebooks(notebooksToRun)
        containers.append(container)
        print("task : ", current_time , str(container), "-->", notebooksToRun['tasks'][0]['params']['file_path'])
        if i%concurrent_containers == 0:
            print(f"Now waiting for {str(len(containers))} tasks to complete before spawning new ones")
            controller.wait_for_tasks_to_complete(containers, 60,30, False)
            print("task : ", containers, " done ")
            containers = []
    
    if len(containers) > 0: 
        print(f"Now waiting for {str(len(containers))} tasks to complete before spawning new ones")
        controller.wait_for_tasks_to_complete(containers, 60,15, False)

In [ ]:
%%time

concurrent_containers = 10
for retry in range(0, 3):
    try: 
        !rm -rf /efs/shared/regression/notebooks/A-LakeCreator/$load_data_notebook/*
        run_glue_table_loading(concurrent_containers)
        executions = controller.get_execution_history("/efs/shared/regression/notebooks/A-LakeCreator", f"{load_data_notebook}.ipynb")  
        display(executions)
        checkNotebooks(executions,5)
        break
    except AssertionError as e:
        print(f"Failed {retry}, lets give one more try")
        #concurrent_containers = concurrent_containers - 1



#### Check that Glue Tables are Created
We will ensure that we now have 7 tables created from our 7 data files that we extracted earlier:

In [ ]:
glue = boto3.client('glue')
res = glue.get_tables(DatabaseName='cms_raw_db')
tables = res['TableList']
raw_count = 0
parq_count = 0
for t in tables:
    if t['Name'].endswith('_raw'):
        raw_count += 1
    else:
        parq_count += 1
    print(t['Name'])
print(f"Total tables: {str(len(tables))}. Raw tables: {raw_count}. Final tables: {parq_count}")
assert raw_count == parq_count and raw_count > 0
!echo "PASSED" >> /efs/shared/regression/PASSED
!ls /efs/shared/regression



# End of notebook

**Congratulations! You have just built you very own Data Lake with countless Amazon Web Service integrations through AWS Orbit Workbench**

<img style="width:100px;height:100px;border:0;" src="https://images-na.ssl-images-amazon.com/images/I/71pHyDfdXwL._SL1500_.jpg" />

In [ ]:
exit()